In [78]:
import pandas as pd
import re

In [97]:
def read_in_compound():
    KEGG_COMPOUND_FILE = '/vortexfs1/home/halexander/KEGG/ligand/compound/compound'
    KEGG_COMPOUND_DICT = {}
    Cnum = {}
    with open (KEGG_COMPOUND_FILE) as f:
        for i, line in enumerate(f):
    #         if i==10000:
    #             break
            if line.startswith('///'):
                for key in ['BRITE', 'DBLINKS','ATOM','BOND','BRACKET','REMARK']:
                    if key in Cnum:
                        Cnum.pop(key)
                KEGG_COMPOUND_DICT[Cnum['ENTRY']]=Cnum
                Cnum={}

            elif re.match('^[A-Z]', line):
                test=line.strip().split()
                cat = test[0]
                value = test[1:]
                if cat == 'ENTRY':
                    value=value[0]
                if cat == 'PATHWAY':
                    value=[value[0]]
                if cat == 'MODULE':
                    value=value[0]    
            else:
                test2 = line.strip().split()
                if cat == 'ENTRY':
                    value+=test2[0]
                elif cat == 'PATHWAY':
                    value+=[test2[0]]
                elif cat == 'MODULE':
                    value+=test2[0]
                else:
                    value+=test2
            Cnum[cat]=value
    return KEGG_COMPOUND_DICT
KEGG_COMPOUND_DICT=read_in_compound()        

In [98]:
KEGG_COMPOUND_DICT['C00719']

{'ENTRY': 'C00719',
 'NAME': ['Betaine;',
  'Trimethylaminoacetate;',
  'Glycine',
  'betaine;',
  'N,N,N-Trimethylglycine;',
  'Trimethylammonioacetate'],
 'FORMULA': ['C5H11NO2'],
 'EXACT_MASS': ['117.079'],
 'MOL_WEIGHT': ['117.1463'],
 'REACTION': ['R02565',
  'R02566',
  'R02821',
  'R07228',
  'R07244',
  'R08211',
  'R08212',
  'R10061',
  'R10062'],
 'PATHWAY': ['map00260', 'map01100', 'map02010'],
 'MODULE': 'M00555',
 'ENZYME': ['1.1.3.17',
  '1.2.1.8',
  '1.21.4.4',
  '2.1.1.5',
  '2.1.1.157',
  '2.1.1.161',
  '2.1.1.162']}

In [125]:
def read_in_reaction():
    KEGG_REACTION_FILE = '/vortexfs1/home/halexander/KEGG/ligand/reaction/reaction'
    KEGG_REACTION_DICT = {}
    Cnum = {}
    with open (KEGG_REACTION_FILE) as f:
        for i, line in enumerate(f):
#             if i==10000:
#                 break
            if line.startswith('///'):
                for key in ['BRITE', 'DBLINKS','ATOM','BOND','BRACKET','COMMENT']:
                    if key in Cnum:
                        Cnum.pop(key)

                KEGG_REACTION_DICT[Cnum['ENTRY']]=Cnum
                Cnum={}

            elif re.match('^[A-Z]', line):
                test=line.strip().split()
                cat = test[0]
                value = test[1:]
                if cat == 'ENTRY':
                    value=value[0]
                if cat == 'NAME':
                    value=[' '.join(value)]
                if cat == 'ORTHOLOGY':
                    value=[value[0]]
                if cat == 'DEFINITION':
                    value=' '.join(value)
                if cat == 'EQUATION':
                    value=' '.join(value)
                if cat == 'PATHWAY':
                    value=[value[0]]
                if cat == 'MODULE':
                    value=value[0]    
            else:
                test2 = line.strip().split()
                if cat == 'ENTRY':
                    value+=test2[0]
                elif cat == 'PATHWAY':
                    value+=[test2[0]]
                elif cat == 'MODULE':
                    value+=test2[0]
                elif cat == 'ORTHOLOGY':
                    value+=[test2[0]]
                else:
                    value+=test2
            Cnum[cat]=value
    return KEGG_REACTION_DICT
KEGG_REACTION_DICT=read_in_reaction()

In [119]:
KEGG_REACTION_DICT['R02566']

{'ORTHOLOGY': ['K00130', 'K14085'],
 'ENTRY': 'R02566',
 'NAME': ['p-Cumic alcohol:NADP+ oxidoreductase'],
 'DEFINITION': 'Betaine aldehyde + NADP+ + H2O <=> Betaine + NADPH + 2 H+',
 'EQUATION': 'C00576 + C00006 + C00001 <=> C00719 + C00005 + 2 C00080',
 'RCLASS': ['RC00001', 'C00005_C00006', 'RC00080', 'C00576_C00719'],
 'ENZYME': ['1.2.1.8'],
 'PATHWAY': ['rn00260', 'rn01100'],
 'MODULE': 'M00555'}

In [151]:
COMPOUND_LIST = ['C06231','C00719', 'C16432','C00334']
for c in COMPOUND_LIST:
    df=pd.DataFrame(columns=['DEFINITION', 'NAME', 'ENZYME', 'ORTHOLOGY'])
    for R in KEGG_COMPOUND_DICT[c]['REACTION']:
        KD=KEGG_REACTION_DICT[R]
        if 'NAME' in KD:
            df.loc[R,'NAME']=KD['NAME']
        if 'ENZYME' in KD:
            df.loc[R,'ENZYME']=KD['ENZYME']
    
        df.loc[R,'DEFINITION']=KD['DEFINITION']
        if 'ORTHOLOGY' in KD:
            df.loc[R,'ORTHOLOGY']=';'.join(KD['ORTHOLOGY'] )
    df.to_csv(c+'-REACTION-INFO.txt', sep='\t')